# 📚 Libraries

In [ ]:
import os
import random
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, map_coordinates

# 🔁 Classic Image Augmentation Functions

In [ ]:
def apply_rotation(img):
    """Apply random rotation to image."""
    rows, cols = img.shape[:2]
    angle = random.uniform(-30, 30)
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    return cv2.warpAffine(img, M, (cols, rows), borderMode=cv2.BORDER_REFLECT)

def apply_zoom(img):
    """Apply random zoom in/out to image."""
    rows, cols = img.shape[:2]
    zoom_factor = random.uniform(0.8, 1.2)
    resized = cv2.resize(img, None, fx=zoom_factor, fy=zoom_factor)
    r_rows, r_cols = resized.shape[:2]

    if r_rows > rows or r_cols > cols:
        start_row = (r_rows - rows) // 2
        start_col = (r_cols - cols) // 2
        return resized[start_row:start_row+rows, start_col:start_col+cols]
    else:
        pad_row = (rows - r_rows) // 2
        pad_col = (cols - r_cols) // 2
        return cv2.copyMakeBorder(resized, pad_row, rows - r_rows - pad_row,
                                  pad_col, cols - r_cols - pad_col,
                                  borderType=cv2.BORDER_REFLECT)

def apply_translation(img):
    """Apply random translation to image."""
    rows, cols = img.shape[:2]
    dx = random.randint(-int(0.1 * cols), int(0.1 * cols))
    dy = random.randint(-int(0.1 * rows), int(0.1 * rows))
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return cv2.warpAffine(img, M, (cols, rows), borderMode=cv2.BORDER_REFLECT)

def apply_flip(img):
    """Apply horizontal flip."""
    return cv2.flip(img, 1)

# 🔊 Spectrogram-Specific Augmentation Functions

In [ ]:
def apply_spectrogram_random_shifts(img):
    """Random pitch and time shift."""
    max_shift = img.shape[1] // 10
    dx = random.randint(-max_shift, max_shift)
    dy = random.randint(-max_shift, max_shift)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), borderMode=cv2.BORDER_REFLECT)

def apply_vtln(img):
    """Simulate Vocal Tract Length Normalization (VTLN) by stretching vertically."""
    scale = random.uniform(0.9, 1.1)
    new_rows = int(img.shape[0] * scale)
    vtln_img = cv2.resize(img, (img.shape[1], new_rows), interpolation=cv2.INTER_LINEAR)
    return cv2.resize(vtln_img, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_LINEAR)

def apply_noise_s(img, noise_percent=0.1, variance=0.1):
    """Add random noise to a portion of the spectrogram pixels."""
    img_noisy = img.copy().astype(np.float32)
    mask = np.random.rand(*img.shape) < noise_percent
    noise = np.random.normal(1.0, variance, img.shape)
    img_noisy[mask] *= noise[mask]
    return np.clip(img_noisy, 0, 255).astype(np.uint8)

# 🧰 List of All Augmentation Functions

In [ ]:
augmentation_functions = [
    apply_rotation,
    apply_zoom,
    apply_translation,
    apply_flip,
    apply_spectrogram_random_shifts,
    apply_vtln,
    apply_noise_s
]

# 🗂️ Parameters and Directory Setup

In [ ]:
source_dir = r"C:\Users\cadur\Downloads\Urbansonic\ESC_50\0_Mels_Gen\DATA\ESC_50_Mel"
output_dir = r"C:\Users\cadur\Downloads\Urbansonic\ESC_50\0_Mels_Gen\DATA\Data_Augmentacion"

# Desired number of images per class
target_counts = {
    "BI": 1296,
    "EM": 800,
    "S": 1200,
    "TM": 1473,
    "VM": 1426
}

classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
print(f"Detected classes: {classes}")

# 🔄 Data Augmentation Pipeline

In [ ]:
for class_name, target_count in target_counts.items():
    print(f"\nProcessing class '{class_name}'...")
    class_src = os.path.join(source_dir, class_name)
    class_dst = os.path.join(output_dir, class_name)
    os.makedirs(class_dst, exist_ok=True)

    images = [f for f in os.listdir(class_src) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    images = sorted(images)
    count = 0

    # Copy original images
    for img_file in images:
        if count >= target_count:
            break
        shutil.copy2(os.path.join(class_src, img_file), os.path.join(class_dst, img_file))
        count += 1

    print(f"Original images copied: {count}")

    if count >= target_count:
        print(f"Class '{class_name}' already meets the target. No augmentation applied.")
        continue

    # Apply augmentations to reach target
    aug_index = 0
    while count < target_count:
        img_filename = random.choice(images)
        img_path = os.path.join(class_src, img_filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error reading image {img_path}. Skipping.")
            continue

        func = augmentation_functions[aug_index % len(augmentation_functions)]
        img_aug = func(img)

        new_filename = f"aug_{count}_{img_filename}"
        new_filepath = os.path.join(class_dst, new_filename)
        ext = os.path.splitext(new_filename)[1].lower()

        if ext in ['.jpg', '.jpeg']:
            cv2.imwrite(new_filepath, img_aug, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
        elif ext == '.png':
            cv2.imwrite(new_filepath, img_aug, [int(cv2.IMWRITE_PNG_COMPRESSION), 3])
        else:
            cv2.imwrite(new_filepath, img_aug)

        count += 1
        aug_index += 1

        if count % 50 == 0:
            print(f"{count} images generated for class '{class_name}'...")

    print(f"Finished class '{class_name}': {count} total images.")

print("\n✅ Data augmentation process completed.")

# 📊 Class Balance Visualization

In [ ]:
train_dir = output_dir
class_names = [folder for folder in os.listdir(train_dir)
               if os.path.isdir(os.path.join(train_dir, folder))]

counts = []
for class_name in class_names:
    class_path = os.path.join(train_dir, class_name)
    image_count = len([f for f in os.listdir(class_path)
                       if os.path.isfile(os.path.join(class_path, f))])
    counts.append(image_count)

fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.tab10(np.linspace(0, 1, len(class_names)))
bars = ax.bar(class_names, counts, color=colors)

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height, f'{int(height)}',
            ha='center', va='bottom', fontsize=12)

ax.set_xlabel('Classes')
ax.set_ylabel('Number of Images')
ax.set_title('Class Balance After Augmentation')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()